In [31]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the path to the dataset
data_dir = 'food-101/food-101'

# ImageDataGenerator for Data Augmentation and Normalization
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% of data used for validation
)

# Create training and validation generators
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',  # Ensure class_mode is 'categorical' for multi-class classification
    subset='training'  # Use this subset for training
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',  # Ensure class_mode is 'categorical' for multi-class classification
    subset='validation'  # Use this subset for validation
)

# Debugging: Print class indices
print(train_generator.class_indices)
print(f"Number of classes: {len(train_generator.class_indices)}")


Found 80800 images belonging to 2 classes.
Found 20200 images belonging to 2 classes.
{'images': 0, 'meta': 1}
Number of classes: 2


In [32]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load the ResNet50 model with pre-trained weights
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False

# Create a new model on top
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dense(len(train_generator.class_indices), activation='softmax')  # Ensure the output layer matches the number of classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Debugging: Print the model summary
model.summary()


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ ?                      │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 23,587,712 (89.98 MB)

In [12]:
# Train the model
history = model.fit(
    train_generator,
    epochs=1,
    validation_data=validation_generator
)


2525/2525 ━━━━━━━━━━━━━━━━━━━━ 3887s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00


In [14]:
# Evaluate on the validation set
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {val_accuracy:.2f}')


632/632 ━━━━━━━━━━━━━━━━━━━━ 589s 932ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Validation Accuracy: 1.00


In [33]:
import cv2
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Example dictionary of average calories per food category
calorie_dict = {
    'apple_pie': 237,
    'baby_back_ribs': 290,
    'baklava': 334,
    'beef_carpaccio': 123,
    'beef_tartare': 175,
    'beet_salad': 74,
    'beignets': 241,
    'bibimbap': 490,
    'bread_pudding': 298,
    'breakfast_burrito': 305,
    'bruschetta': 175,
    'caesar_salad': 190,
    'cannoli': 276,
    'caprese_salad': 250,
    'carrot_cake': 415,
    'ceviche': 100,
    'cheesecake': 321,
    'cheese_plate': 200,
    'chicken_curry': 280,
    'chicken_quesadilla': 350,
    'chicken_wings': 430,
    'chocolate_cake': 352,
    'chocolate_mousse': 350,
    'churros': 237,
    'clam_chowder': 200,
    'club_sandwich': 320,
    'crab_cakes': 220,
    'creme_brulee': 330,
    'croque_madame': 300,
    'cup_cakes': 305,
    'deviled_eggs': 200,
    'donuts': 452,
    'dumplings': 250,
    'edamame': 121,
    'eggs_benedict': 400,
    'escargots': 220,
    'falafel': 330,
    'filet_mignon': 679,
    'fish_and_chips': 600,
    'foie_gras': 445,
    'french_fries': 365,
    'french_onion_soup': 369,
    'french_toast': 450,
    'fried_calamari': 150,
    'fried_rice': 228,
    'frozen_yogurt': 159,
    'garlic_bread': 206,
    'gnocchi': 250,
    'greek_salad': 210,
    'grilled_cheese_sandwich': 320,
    'grilled_salmon': 400,
    'guacamole': 160,
    'gyoza': 180,
    'hamburger': 354,
    'hot_and_sour_soup': 70,
    'hot_dog': 300,
    'huevos_rancheros': 360,
    'hummus': 166,
    'ice_cream': 207,
    'lasagna': 135,
    'lobster_bisque': 270,
    'lobster_roll_sandwich': 436,
    'macaroni_and_cheese': 450,
    'macarons': 70,
    'miso_soup': 40,
    'mussels': 146,
    'nachos': 550,
    'omelette': 154,
    'onion_rings': 276,
    'oysters': 50,
    'pad_thai': 300,
    'paella': 300,
    'pancakes': 227,
    'panna_cotta': 282,
    'peking_duck': 337,
    'pho': 400,
    'pizza': 266,
    'pork_chop': 340,
    'poutine': 740,
    'prime_rib': 400,
    'pulled_pork_sandwich': 440,
    'ramen': 436,
    'ravioli': 320,
    'red_velvet_cake': 370,
    'risotto': 380,
    'samosa': 262,
    'sashimi': 231,
    'scallops': 75,
    'seaweed_salad': 70,
    'shrimp_and_grits': 250,
    'spaghetti_bolognese': 350,
    'spaghetti_carbonara': 360,
    'spring_rolls': 100,
    'steak': 679,
    'strawberry_shortcake': 275,
    'sushi': 200,
    'tacos': 226,
    'takoyaki': 110,
    'tiramisu': 240,
    'tuna_tartare': 190,
    'waffles': 291
}

# Function to estimate calories based on the predicted class
def estimate_calories(predicted_class):
    return calorie_dict.get(predicted_class, 0)  # Return 0 if the class is not found

# Function to preprocess image
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Create batch axis
    return img_array / 255.0  # Normalize the image

# Function to predict food item and estimate calories
def predict_from_image(image_path):
    img_array = preprocess_image(image_path)
    predictions = model.predict(img_array)
    predicted_index = np.argmax(predictions)
    predicted_class = list(train_generator.class_indices.keys())[predicted_index]
    calories = estimate_calories(predicted_class)
    return predicted_class, calories

# Function to capture image from webcam and predict
def predict_from_camera(model, class_indices):
    cap = cv2.VideoCapture(0)  # Open the webcam (usually device 0)
    if not cap.isOpened():
        raise ValueError("Could not open the webcam.")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture image. Please try again.")
            continue
        
        cv2.imshow('Press "c" to capture', frame)
        if cv2.waitKey(1) & 0xFF == ord('c'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

    # Save the captured image
    camera_image_path = 'captured_image.jpg'
    if frame is not None and frame.size != 0:
        cv2.imwrite(camera_image_path, frame)
    else:
        raise ValueError("Captured image is empty.")

    # Predict from the captured image
    return predict_from_image(camera_image_path, model, class_indices)

# Main function to predict calories
def predict_calories(input_type, input_value=None):
    if input_type == 'name':
        calories = estimate_calories(input_value)
        return input_value, calories
    elif input_type == 'image':
        if input_value is None:
            raise ValueError("For image input, input_value must be the path to the image.")
        return predict_from_image(input_value)
    elif input_type == 'camera':
        return predict_from_camera()
    else:
        raise ValueError("Invalid input type. Choose from 'name', 'image', or 'camera'.")


In [34]:
food_name = 'apple_pie'
predicted_class, calories = predict_calories('name', food_name)
print(f'Predicted food item: {predicted_class}, Estimated calories: {calories}')

Predicted food item: apple_pie, Estimated calories: 237
